In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

import requests
import json

import string
import math

from collections import Counter
import random

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /cfs/home/u021320/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
characters = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green', 'others']

#test set 1 = season 8

with open('sets/train_set1.json') as f:
    train_set1 = json.load(f)

with open('sets/train_set2.json') as f:
    train_set2 = json.load(f)

#CHANGE HERE
train_set = train_set1
#number = 1

all_lines = []

#corpus
for line in train_set:
    all_lines.append(line['transcript'])

In [3]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit(all_lines)

feature_names = vectorizer.get_feature_names_out()


#tf-idf for each character
for character in characters:
    character_lines = []
    for line in train_set:
        if character != 'others' and character in line['speakers']:
            character_lines.append(line['transcript'])
        elif character == 'others' and line['speakers'][0] not in characters:
            character_lines.append(line['transcript'])


    tf_idf = vectorizer.transform(character_lines)

    dense = tf_idf.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)

    values = []

    for sentence in dense:
        values.append(sentence.sum())

    
    #associate values with sentences
    sentences = []
    for i in range(len(character_lines)):
        sentences.append({'sentence': character_lines[i], 'value': values[i]})
    
    #sort sentences by value
    sentences = sorted(sentences, key = lambda i: i['value'], reverse=True)

    #change x for number of lines 
    values = [100,150,500,1000,2000]

    for x in values:

        f = open("embeddings" + "/" + character + str(x)+ ".txt", "w")
        for line in sentences[:x]:
            f.write(line['sentence']+"\n")
        f.close()

In [4]:
def tf_idf_calculator(line, character_lines, idf):

    counts = 0

    for sentence in character_lines:
        if line in sentence:
            counts += 1

    tf = counts / len(character_lines)

    tf_idf = tf * idf[line]

    return tf_idf

In [4]:
idf = {}

all_lines_set = [sentence.translate(str.maketrans('', '', string.punctuation)).lower() for sentence in all_lines]

all_lines_set_no_stopwords = []

for line in all_lines_set:
    if line not in stopwords.words('english'):
        all_lines_set_no_stopwords.append(line)


for line in all_lines_set:
    counts = 0
    for sentence in all_lines_set:
        if line in sentence:
            counts += 1
    
    idf[line] = math.log(len(all_lines) / (1 + counts))

#tf-idf for each character
for character in characters:
    character_lines = []
    for line in train_set:
        if character != 'others' and character in line['speakers']:
            character_lines.append(line['transcript'])
        elif character == 'others' and line['speakers'][0] not in characters:
            character_lines.append(line['transcript'])
    
    tf_idfs = []

    character_lines_processed = []

    for line in character_lines:
        line = line.translate(str.maketrans('', '', string.punctuation)).lower()
        if line not in stopwords.words('english'):
            character_lines_processed.append(line)

    for line in character_lines_processed:
        tf_idfs.append(tf_idf_calculator(line, character_lines_processed, idf))

    #associate values with sentences
    sentences = []
    for i in range(len(character_lines_processed)):
        sentences.append({'sentence': character_lines_processed[i], 'value': tf_idfs[i]})
    
    #sort sentences by value
    sentences = sorted(sentences, key = lambda i: i['value'], reverse=True)

    sentences_only = [sentence['sentence'] for sentence in sentences]

    #remove duplicates
    sentences_only = list( dict.fromkeys(sentences_only) )

    #change x for number of lines 
    values = [5,10,20,50,100,150,200,500,1000,2000,5000, 6000]

    for x in values:

        f = open("embeddings" + "/" + character + str(x)+ ".txt", "w")
        for line in sentences_only[:x]:
            f.write(line+"\n")
        f.close()

In [5]:
def calculate_custom_idf(all_lines):
    idf = {}
    total_documents = len(all_lines)

    term_document_count = {}
    for line in all_lines:
        counts = 0
        for sentence in all_lines_set:
            if line in sentence:
                counts += 1
        term_document_count[line] = counts

    # Compute the IDF values with customization
    for line, count in term_document_count.items():

        idf[line] = math.log(total_documents / (count + 1))  # Adding 1 to avoid division by zero

        # Customization: Lower the IDF for frequently occurring terms
        if count > total_documents * 0.007:  # Adjust the threshold as needed
            idf[line] *= 0.05  # Adjust the penalty factor as needed

    return idf

In [6]:
idf = {}

all_lines_set = [sentence.translate(str.maketrans('', '', string.punctuation)).lower() for sentence in all_lines]

all_lines_set_no_stopwords = []

for line in all_lines_set:
    if line not in stopwords.words('english'):
        all_lines_set_no_stopwords.append(line)

idf = calculate_custom_idf(all_lines_set_no_stopwords)


#tf-idf for each character
for character in characters:
    character_lines = []
    for line in train_set:
        if character != 'others' and character in line['speakers']:
            character_lines.append(line['transcript'])
        elif character == 'others' and line['speakers'][0] not in characters:
            character_lines.append(line['transcript'])
    
    tf_idfs = []

    character_lines_processed = []

    for line in character_lines:
        line = line.translate(str.maketrans('', '', string.punctuation)).lower()
        if line not in stopwords.words('english'):
            character_lines_processed.append(line)

    for line in character_lines_processed:
        tf_idfs.append(tf_idf_calculator(line, character_lines_processed, idf))


    #associate values with sentences
    sentences = []
    for i in range(len(character_lines_processed)):
        sentences.append({'sentence': character_lines_processed[i], 'value': tf_idfs[i]})
    
    #sort sentences by value
    sentences = sorted(sentences, key = lambda i: i['value'], reverse=True)

    sentences_only = [sentence['sentence'] for sentence in sentences]

    #remove duplicates
    sentences_only = list( dict.fromkeys(sentences_only) )

    #change x for number of lines 
    values = [100,150,500,1000,2000]

    for x in values:

        f = open("embeddings" + "/" + character + str(x)+ ".txt", "a")
        for line in sentences_only[:x]:
            f.write(line+"\n")
        f.close()